# Categorical Naive Bayes Classifier 

Naive bayes models are a broad family of supervised machine learning models that can be used for both classification and regression tasks. They are all based on Bayes' theorem with the so-called _"naive assumption"_ which assumes that all features are indepedent from each other. Specifically, in this notebook we will implement a categorical naive Bayes classifier which assumes that the features can only take on discrete values. These values do not represent a count, instead they each discrete value is associated with a category or label. A typical use for this type of model is in the detection of spam emails. Below is data collected on old spam SMS texts,

In [3]:
# Imports,
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd

# Loading data,
data = np.load("dataset.npz", allow_pickle=True)
X, y, feature_labels = data["features"], data["targets"], data["feature_labels"]

# Creating dataframe,
df = pd.DataFrame(data=X, columns=feature_labels)
df["target"] = y
df

,call,to,free,txt,your,or,now,mobile,claim,text,...,å2000,mins,landline,shows,camera,16,box,only,holiday,target
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
5568,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Each feature "call", "to", "free" are words that are that may or may not be in an SMS text message. Therefore, each feature $x_j$ can only have a value of 0 (the word is not in the text) or 1 (the word appears in the text atleast once). Our features are encoded in a feature vector $X = [x_1, x_2, x_3, ..., x_M]$ with $M=50$ number of features and $C=2$ number of classes such that $y=0$ for non-spam and $y=1$ for spam texts. Our starting point for a naive Bayes model is Bayes' theorem which states that, 

$$P(A|B) = \frac{P(A)P(B|A)}{P(B)}$$

Essentially, we can find the probability of an event $A$ occuring given that $B$ has occured (posterior probability) based on known information such as the likelihood $P(B|A)$, prior $P(A)$ and marginal $P(B)$ probabilities. In our machine learning context, we let $B$ represent the event that the feature vector $X$ is observed and event $A$ that the class label $y$ is the associated classification. 

$$P(y|X) = \frac{P(y)P(X|y)}{P(X)}$$

The event that the feature vector $X$ is observed can be thought of as the independent events of $x_1$, $x_2$, $x_3$, ..., $x_M$ being observed simulataneously. This assumption is why we call the model is called a **naive** Bayes model. With this, we are able to write,

$$P(y|X) = \frac{P(y)}{P(X)} \prod_{j=1}^{M} P(x_j|y) $$

from the likelihood. We may also rewrite the the marginal probability $P(X)$ using an identity from probability theory,

$$P(y|X) = \frac{P(y)\prod_{j=1}^{M} P(x_j|y)}{\sum_{k=1}^{C} P(X|y_k)P(y_k)}$$

Once again, using the _"naive assumption"_,

$$P(y|X) = \frac{P(y)\prod_{j=1}^{M} P(x_j|y)}{\sum_{k=1}^{C} P(y_k) \prod_{j=1}^{M} P(x_j|y_k)}$$

Notice that the margin probabilitiy, our denominator, is simply a normalisation factor since it runs over all possible class labels. Let us call this normalisation factor $Z = \sum_{k=1}^{C} P(y_k) \prod_{j=1}^{M} P(x_j|y_k)$. The probability of observing the class label $y=y_l$ given an associated feature vector $X$ has been measured is,

$$P(y=y_l|X) = \frac{1}{Z}P(y=y_l)\prod_{j=1}^{M} P(x_j|y=y_k)$$

This formula is basically the entire model summarised into a single equation. When making a prediction $\hat{y}$, the model computes $P(y|X)$ (or a another quantity proportional to it) for all class labels $y_1$, $y_2$, ..., $y_C$. The class label with the largest probability is assigned as the predicted class such that, 

$$\hat{y} = \text{argmax}_{y} \left[ P(y|X) \right]$$

Since we are dealing with discrete features, computing Z is quite straight-foward. For $P(y=y_k)$ we simply have $P(y=y_k)=N_k/N$ where $N_k$ is the nymber of examples with the label $y=y_l$ and $N$ is the total number of examples. Calculating $P(x_j|y_k)$ is a little more complicated,

$$P(x_j = \nu|y_k) = \frac{N_{\nu j k} + \alpha}{N_k + n_j \alpha}$$

In which, $N_{\nu j k}$ is the number of samples with the class label $y=y_k$ and the feature value $x_j = \nu$ (for some $j = 1, 2, 3, ..., M$), $n_j$ is the number of features in $X$ and $\alpha$ with is the Laplace smoothing parameter (usually $0< \alpha \leq 1$). The smoothing parameter $\alpha$ ensures that we do not encounter zero probabilites when $N_{\nu j k}=0$ which breaks our model.

# Model

Below, we implement a very basic MNB. The essential idea is that we want to reserve the majority of the computation for when we create and fit the model rather than when we need to calculate predictios. This will save us time and computational resources. We want our model calculate $P(y|X)$ for all possible labels $y$ given some observed $X$. Let the all $P(y|X)$ be stored in some array $\mathbf{P}$ such that,

 $\mathbf{P} = \left[P(y=y_1|X), P(y=y_2|X), ..., P(y=y_C|X) \right]$  

We also define the following arrays to store the likelihood and prior probabilities, 

$\mathbf{P_{Xy}} = \left[P(X|y=y_1), P(X|y=y_1), ..., P(X|y=y_C) \right]$

$\mathbf{P_{y}} = \left[P(y=y_1), P(y=y_2), ..., P(y=y_3) \right]$

With this formulation, our probabilites are calculated via,

$\mathbf{P} = \frac{\mathbf{P_{y}} \mathbf{P_{Xy}}}{ \mathbf{P_{y}} \cdot \mathbf{P_{Xy}}}$

In which, $Z = \mathbf{P_{y}} \cdot \mathbf{P_{Xy}}$. In the model fitting, we create a so-called likelihood tensor (3D array) whose elements are $P(x_j|y)$. Let this tensor be denoted by $\mathbf{T}$ with the elements $T_{kj \nu} = P(x_j=\nu|y=y_k)$. From $\mathbf{T}$, we are able to simply pick out its elements and reconstruct $\mathbf{P_{Xy}}$ and $\mathbf{P_{y}}$ to compute the probabilities $\mathbf{P}$. Note that $\mathbf{T}$ has the dimensions $[C, M, n_j]$ where $n_j$ is the number of unique values a feature $x_j$ could have ($n_j=2$ for all $j$ in our case).

In [11]:
class CategoricalNaiveBayes():
    """
    A simple implementation of the Categorical Naive Bayes classifier for discrete/categorical feature data.

    This model assumes that each feature follows a multinomial distribution conditioned on the class label.
    It applies Laplace smoothing controlled by the hyperparameter `alpha`.

    Attributes:
        X (ndarray): The training feature matrix of shape (n_samples, n_features).
        y (ndarray): The target labels corresponding to `X`, of shape (n_samples,).
        alpha (float): Laplace smoothing parameter (must be >= 0).
    """

    def __init__(self, alpha):
        """Constructor method. Class variables are stored or computed."""

        # Creating class variables,
        self.X, self.y = None, None
        self.nX_possible_values, self.y_possible_values = None, None
        self.alpha = alpha
        
        # Placeholders,
        self.Py_vector = None
        self.likelihoods = []

    def fit(self, X, y):
        """Bulk of the required calculation is performed by fitting the model. Specifically, our probability vectors are 
        computed."""

        # Assigning class variables,
        self.X, self.y = X, y

        # Computing possible values for features and classes,
        self.nX_possible_values = np.apply_along_axis(lambda col: len(np.unique(col)), axis=0, arr=X)
        self.n_features = X.shape[1]
        self.feature_values = [np.unique(X[:, i]) for i in range(self.n_features)]

        # Computing P(y) for all class labels,
        self.y_possible_values, y_counts = np.unique(y, return_counts=True)
        self.Py_vector = y_counts/len(y) # <-- Formula 
        self.n_classes = len(self.y_possible_values)
        self.max_categories = max(self.nX_possible_values)

        # Creating likelihood tensor,
        self.likelihoods = np.full((self.n_classes, self.n_features, self.max_categories), fill_value=1, dtype=float)

        # Computing all P(x_i|y=yk) for lik,
        for cls_idx, cls in enumerate(self.y_possible_values):

            cls_idxs = np.where(self.y==cls)[0]
            X_given_y = X[cls_idxs]

            # Double loop over features and then possible feature values,
            for feature_idx in range(self.n_features):

                # P(x_i|y=yk) for all values x_i can take,
                for feature_value in self.feature_values[feature_idx]:
                    n_instances = len(np.where(X_given_y.T[feature_idx]==feature_value)[0])
                    n_j = self.nX_possible_values[feature_idx]
                    Pxiy = (n_instances + self.alpha)/(len(cls_idxs) + n_j*self.alpha)
                    
                    # Adding to likelihood tensor,
                    self.likelihoods[cls_idx, feature_idx, feature_value] = Pxiy

    def predict(self, X_sample):

        # Initialising arrays,
        self.probs = []
        self.PXy_vector = []

        for class_idx, cls in enumerate(self.y_possible_values):

            # Extracting required P(y=y_l),
            Py = self.Py_vector[class_idx]

            # Computing P(X|y=y_l),
            PXy = 1 # <-- Placeholder value

            for feature_idx, feature in enumerate(X_sample):
                PXiy = self.likelihoods[class_idx][feature_idx][feature] # <-- Extracted from pre-computed matrix
                PXy = PXy*PXiy
            self.PXy_vector.append(PXy)

        # Computing P(X) from the vectors (normalisation),
        PX = np.dot(self.PXy_vector, self.Py_vector)

        # Final calculation,
        self.probs = (self.Py_vector*self.PXy_vector)/PX

        # Prediction as the most likely probability,
        pred = self.y_possible_values[np.argmax(self.probs)]

        return pred

    def score(self, X, y):
        """The model prediction is taken as the class with the most likely probability."""

        # Counting number of correct predictions,
        correct = 0
        for i, X_sample in enumerate(X):
            pred = self.predict(X_sample)
            if pred == y[i]:
                correct += 1

        # Computing accuracy,
        accuracy = correct/len(y)

        return accuracy

Our model has an accuracy of roughly around 92-95% on the testing dataset,

In [9]:
# HYPERPARAMETERS,
ALPHA = 0.1

# Creating data split,
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Creating and fitting model on training data,
clf = CategoricalNaiveBayes(alpha=ALPHA)
clf.fit(X_train, y_train)

# Computing accuracy on testing data,
clf.score(X_test, y_test)

0.9551971326164874

When benchmarking our model in comparision to scikit-learn's implementation of the categorical naive Bayes model, the results are comparable. 

In [12]:
# Creating and fitting model on training data,
clf = CategoricalNB(alpha=ALPHA)
clf.fit(X_train, y_train)

# Computing accuracy on testing data,
clf.score(X_test, y_test)

0.9551971326164874

## Optimisation

However, there are differences between our implementation and that of scikit-learn's. Firstly, to save computational resources, $Z = P(X)$ is not calculated since it is just a normalisation constant. Moroever, our implementation allows for the possibility of floating-point underflow when calculating $P(X|y)$ since $P(x_1|y)P(x_2|y)P(x_3|y)...$ may be an extremely small value. To prevent this, the logarithmic probabilities $\log{[P(y|X)]}$ are calculated instead. Mathematically,

$$P(y=y_l|X) = \frac{1}{Z}P(y=y_l)\prod_{j=1}^{M} P(x_j|y=y_k)$$

$$P(y=y_l|X) \sim P(y=y_l)\prod_{j=1}^{M} P(x_j|y=y_k)$$

$$\log{[P(y=y_l|X)]} \sim \log{[P(y=y_l)]} + \sum_{j=1}^{M} \log{[P(x_j|y=y_k)]}$$

In our array formalism, we have, 

$$\log{(\mathbf{P})} \sim \log{(\mathbf{P_y})} + \log{(\mathbf{P_{Xy}})}$$


In [13]:
class CategoricalNaiveBayes():
    """
    A simple implementation of the Categorical Naive Bayes classifier for discrete/categorical feature data.

    This model assumes that each feature follows a multinomial distribution conditioned on the class label.
    It applies Laplace smoothing controlled by the hyperparameter `alpha`.

    Attributes:
        X (ndarray): The training feature matrix of shape (n_samples, n_features).
        y (ndarray): The target labels corresponding to `X`, of shape (n_samples,).
        alpha (float): Laplace smoothing parameter (must be >= 0).
    """

    def __init__(self, alpha):
        """Constructor method. Class variables are stored or computed."""

        # Creating class variables,
        self.X, self.y = None, None
        self.nX_possible_values, self.y_possible_values = None, None
        self.alpha = alpha
        
        # Placeholders,
        self.Py_vector = None
        self.likelihoods = []

    def fit(self, X, y):
        """Bulk of the required calculation is performed by fitting the model. Specifically, our probability vectors are 
        computed."""

        # Assigning class variables,
        self.X, self.y = X, y

        # Computing possible values for features and classes,
        self.nX_possible_values = np.apply_along_axis(lambda col: len(np.unique(col)), axis=0, arr=X)
        self.n_features = X.shape[1]
        self.feature_values = [np.unique(X[:, i]) for i in range(self.n_features)]

        # Computing P(y) for all class labels,
        self.y_possible_values, y_counts = np.unique(y, return_counts=True)
        self.Py_vector = y_counts/len(y) # <-- Formula 
        self.n_classes = len(self.y_possible_values)
        self.max_categories = max(self.nX_possible_values)

        # Creating likelihood tensor,
        self.likelihoods = np.full((self.n_classes, self.n_features, self.max_categories), fill_value=1, dtype=float)

        # Computing all P(x_i|y=yk) for lik,
        for cls_idx, cls in enumerate(self.y_possible_values):

            cls_idxs = np.where(self.y==cls)[0]
            X_given_y = X[cls_idxs]

            # Double loop over features and then possible feature values,
            for feature_idx in range(self.n_features):

                # P(x_i|y=yk) for all values x_i can take,
                for feature_value in self.feature_values[feature_idx]:
                    n_instances = len(np.where(X_given_y.T[feature_idx]==feature_value)[0])
                    n_j = self.nX_possible_values[feature_idx]
                    Pxiy = (n_instances + self.alpha)/(len(cls_idxs) + n_j*self.alpha)
                    
                    # Adding to likelihood tensor,
                    self.likelihoods[cls_idx, feature_idx, feature_value] = Pxiy

    def predict(self, X_sample, return_probs=False):

        # Initialising arrays,
        self.Log_probs = []
        self.probs = None

        for class_idx, cls in enumerate(self.y_possible_values):

            # Extracting required P(y=y_l),
            Log_Py = np.log(self.Py_vector[class_idx])

            # Computing P(X|y=y_l),
            Log_PXy = 0 # <-- Placeholder value

            for feature_idx, feature_value in enumerate(X_sample):
                Log_PXiy = np.log(self.likelihoods[class_idx][feature_idx][feature_value]) # <-- Extracted from pre-computed matrix
                Log_PXy += Log_PXiy

            # Final calculation,
            Log_PykX = Log_Py + Log_PXy
            self.Log_probs.append(Log_PykX)

        if return_probs:

            # Recovering PXy_vector,
            PXy_vector = np.exp(self.Log_probs)
            
            # Computing P(X),
            PX = np.dot(self.Py_vector, PXy_vector)

            # Calculating probabilities, 
            self.probs = (self.Py_vector*PXy_vector)/PX

            return self.probs
        else:
            # Prediction as the most likely probability,
            pred = self.y_possible_values[np.argmax(self.Log_probs)]

            return pred

    def score(self, X, y):
        """The model prediction is taken as the class with the most likely probability."""

        # Counting number of correct predictions,
        correct = 0
        for i, X_sample in enumerate(X):
            pred = self.predict(X_sample)
            if pred == y[i]:
                correct += 1

        # Computing accuracy,
        accuracy = correct/len(y)

        return accuracy